In [ ]:
!pip install yargy
!pip install ipymarkup==0.5.0

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving шекспир.txt to шекспир (1).txt
User uploaded file "шекспир.txt" with length 208889 bytes


In [ ]:
text = open('шекспир.txt', 'r', encoding="utf-8")
l = [line.strip() for line in text]
text.close()
l=str(l)

In [ ]:
from IPython.display import display
from ipymarkup import show_markup, AsciiMarkup

from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, in_, dictionary,
    type, gram, normalized
)
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute

In [ ]:
from yargy.tokenizer import MorphTokenizer

TOKENIZER = MorphTokenizer()
print(list(TOKENIZER('Ромео')))

[MorphToken(value='Ромео', span=[0, 5), type='RU', forms=[Form('ромео', Grams(Fixd,NOUN,Name,anim,masc,nomn,sing)), Form('ромео', Grams(Fixd,NOUN,Name,anim,gent,masc,sing)), Form('ромео', Grams(Fixd,NOUN,Name,anim,datv,masc,sing)), Form('ромео', Grams(Fixd,NOUN,Name,accs,anim,masc,sing)), Form('ромео', Grams(Fixd,NOUN,Name,ablt,anim,masc,sing)), Form('ромео', Grams(Fixd,NOUN,Name,anim,loct,masc,sing)), Form('ромео', Grams(Fixd,NOUN,Name,anim,masc,nomn,plur)), Form('ромео', Grams(Fixd,NOUN,Name,anim,gent,masc,plur)), Form('ромео', Grams(Fixd,NOUN,Name,anim,datv,masc,plur)), Form('ромео', Grams(Fixd,NOUN,Name,accs,anim,masc,plur)), Form('ромео', Grams(Fixd,NOUN,Name,ablt,anim,masc,plur)), Form('ромео', Grams(Fixd,NOUN,Name,anim,loct,masc,plur))])]


In [ ]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


romeo = rule(
    and_(
        gram('ADJF'),  
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'Ромео'
    })
)


parser = Parser(romeo)
for match in parser.findall(l):
    print([_.value for _ in match.tokens])

['дрянном', 'Ромео']
['бывшая', 'Ромео']
['этого', 'Ромео']
['Бедный', 'Ромео']
['молодого', 'Ромео']
['молодой', 'Ромео']
['милый', 'твой', 'Ромео']
['твой', 'Ромео']
['моим', 'Ромео']
['мой', 'Ромео']
['двадцатилетний', 'Ромео']


In [ ]:
jul = rule(
    and_(
    dictionary({
        'Джульетта'
    })),
    gram('VERB'),
)


parser = Parser(jul)
for match in parser.findall(l):
    print([_.value for _ in match.tokens])

['Джульетта', 'уходит']
['Джульетте', 'говоришь']
['Джульетта', 'померла']
['Джульетта', 'умерла']
['Джульетта', 'может']
['Джульетта', 'пробуждается']


In [ ]:
kill = rule(
    and_(
    dictionary({
        'убил'
    })),
    gram('NOUN'),
)


parser = Parser(kill)
for match in parser.findall(l):
    print([_.value for _ in match.tokens])

['Убей', 'луну']
['Убит', 'Ромео']
['убил', 'разбойник']
['Убит', 'Тибальт']
['Убил', 'Тибальта']
['Убил', 'ли']
['убить', 'Тибальт']
['убил', 'Тибальта']
['Убит', 'Тибальт']


In [ ]:
def show_matches(rule, *l):
    parser = Parser(rule) #инициализируем Parser
    for line in l:
        matches = parser.findall(line) #у парсера есть несколько методов, findall - возвращает список matches
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            display(facts)

In [ ]:
Romeo_Relations = fact(
    'Romeo_Relations',
    ['type']
)


FAMILY = morph_pipeline([
    'Монтекки',
    'Абрам',
    'Бенволио',
]).interpretation(
    Romeo_Relations.type.const('семья')
)

ENEMY = morph_pipeline([
    'Капулетти',
    'Тибальт',
    'Парис',
]).interpretation(
    Romeo_Relations.type.const('враг')
)



RR = rule(
    or_(FAMILY, ENEMY),
    
).interpretation(
    Romeo_Relations
)
show_matches(
    RR,
    'И ты тут в красном саване, Тибальт?',
    'Спать на траве и холодно и сыро. Идём, Бенволио.'
)

BoxMarkup('И ты тут в красном саване, Тибальт?', [Span(27, 34, None)])

Romeo_Relations(
    type='враг'
)

BoxMarkup('Спать на траве и холодно и сыро. Идём, Бенволио.',
          [Span(39, 47, None)])

Romeo_Relations(
    type='семья'
)